In [4]:
from keras.layers import Conv2D, UpSampling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf

In [5]:
path = 'dataset/'

train_datagen = ImageDataGenerator(rescale=1. / 255)
train = train_datagen.flow_from_directory(path,  target_size=(256, 256),  batch_size=340, class_mode=None)

Found 60 images belonging to 1 classes.


In [6]:
X =[]
Y =[]
for img in train[0]:
    try:
        lab = rgb2lab(img)
        X.append(lab[:,:,0]) 
        Y.append(lab[:,:,1:] / 128)
    except:
        print('error')

In [7]:
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)

(60, 256, 256, 1)
(60, 256, 256, 2)


In [8]:
# encoder network

model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2, input_shape=(256, 256, 1)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(512, (3,3), activation='relu', padding='same'))
model.add(Conv2D(256, (3,3), activation='relu', padding='same'))

# decoder network
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))

In [9]:
model.compile(optimizer='adam', loss='mse' , metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 256)       295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 256)       590080    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 512)       1180160   
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 512)       2

In [ ]:
model.fit(X, Y, validation_split = 0.1, epochs = 100, batch_size = 16)

Epoch 1/100
4/4 [==============================] - 1s 222ms/step - loss: 1.0208 - accuracy: 0.4632 - val_loss: 1.0777 - val_accuracy: 0.4525
Epoch 2/100
4/4 [==============================] - 1s 277ms/step - loss: 1.0208 - accuracy: 0.4632 - val_loss: 1.0777 - val_accuracy: 0.4525
Epoch 3/100
4/4 [==============================] - 1s 278ms/step - loss: 1.0208 - accuracy: 0.4632 - val_loss: 1.0777 - val_accuracy: 0.4525
Epoch 4/100
4/4 [==============================] - 1s 279ms/step - loss: 1.0208 - accuracy: 0.4632 - val_loss: 1.0777 - val_accuracy: 0.4525
Epoch 5/100
4/4 [==============================] - 1s 280ms/step - loss: 1.0208 - accuracy: 0.4632 - val_loss: 1.0777 - val_accuracy: 0.4525
Epoch 6/100
4/4 [==============================] - 1s 279ms/step - loss: 1.0208 - accuracy: 0.4632 - val_loss: 1.0777 - val_accuracy: 0.4525
Epoch 7/100
4/4 [==============================] - 1s 279ms/step - loss: 1.0208 - accuracy: 0.4632 - val_loss: 1.0777 - val_accuracy: 0.4525
Epoch 8/100
4

In [ ]:
# saving the trained model for future use

model.save('colorize_autoencoder.h5')

In [ ]:
model = tf.keras.models.load_model('colorize_autoencoder.h5', custom_objects=None, compile=True)

In [3]:
import cv2

img1_color=[]
img1 = img_to_array(load_img('sunset.png'))
img1 = resize(img1 ,(256,256))
img1_color.append(img1)

img1_color = np.array(img1_color, dtype=float)
img1_color = rgb2lab(1.0/255*img1_color)[:,:,:,0]
img1_color = img1_color.reshape(img1_color.shape+(1,))

output1 = model.predict(img1_color)
output1 = output1*128

result = np.zeros((256, 256, 3))
result[:,:,0] = img1_color[0][:,:,0]
result[:,:,1:] = output1[0]

cv2.imshow('Gray Scale image', cv2.imread('sunset.png'))
cv2.waitKey(0)
cv2.imshow('Colored Image', lab2rgb(result))
cv2.waitKey(0)
cv2.destroyAllWindows()